In [ ]:
from operator import itemgetter
import matplotlib.pyplot as plt
from tqdm.notebook import trange
import firedrake
from firedrake import (
    min_value, max_value, exp, sqrt, inner, sym, tr, grad, Constant, dx, ds
)
import irksome
from irksome import Dt
import icepack
from icepack.constants import (
    weertman_sliding_law,
    glen_flow_law,
    gravity as g,
    ice_density as ρ_I,
    water_density as ρ_W,
)

Make the mesh and function spaces.

In [ ]:
lx, ly = 640e3, 80e3
ny = 20
nx = int(lx / ly) * ny
area = lx * ly

mesh = firedrake.RectangleMesh(nx, ny, lx, ly, name="mesh")
Q = firedrake.FunctionSpace(mesh, "CG", 1)
V = firedrake.VectorFunctionSpace(mesh, "CG", 1)

Make the bed topography.

In [ ]:
Lx, Ly = Constant(lx), Constant(ly)

def mismip_bed(mesh):
    x, y = firedrake.SpatialCoordinate(mesh)

    x_c = Constant(300e3)
    X = x / x_c

    B_0 = Constant(-150)
    B_2 = Constant(-728.8)
    B_4 = Constant(343.91)
    B_6 = Constant(-50.57)
    B_x = B_0 + B_2 * X**2 + B_4 * X**4 + B_6 * X**6

    f_c = Constant(4e3)
    d_c = Constant(500)
    w_c = Constant(24e3)

    B_y = d_c * (
        1 / (1 + exp(-2 * (y - Ly / 2 - w_c) / f_c)) +
        1 / (1 + exp(+2 * (y - Ly / 2 + w_c) / f_c))
    )

    z_deep = Constant(-720)
    
    return max_value(B_x + B_y, z_deep)

In [ ]:
z_b = firedrake.Function(Q).interpolate(mismip_bed(mesh))

Make the friction law -- use regularized Coulomb friction instead of Weertman.

In [ ]:
m = Constant(weertman_sliding_law)

def friction(**kwargs):
    variables = ("velocity", "thickness", "surface", "friction")
    u, h, s, C = map(kwargs.get, variables)

    p_W = ρ_W * g * max_value(0, -(s - h))
    p_I = ρ_I * g * h
    N = max_value(0, p_I - p_W)
    τ_c = N / 2

    u_c = (τ_c / C) ** m
    u_b = sqrt(inner(u, u))

    return τ_c * (
        (u_c**(1 / m + 1) + u_b**(1 / m + 1))**(m / (m + 1)) - u_c
    )

Form the remaining parts of the physics equations.

In [ ]:
n = Constant(glen_flow_law)

def viscosity(**kwargs):
    names = ("velocity", "thickness", "fluidity")
    u, h, A = itemgetter(*names)(kwargs)
    ε = sym(grad(u))
    εCε = (inner(ε, ε) + tr(ε)**2) / 2
    return 2 * n / (n + 1) * h * A ** (-1 / n) * εCε ** ((n + 1) / (2 * n))

def gravity(**kwargs):
    names = ("thickness", "surface", "velocity")
    h, s, u = itemgetter(*names)(kwargs)
    return -ρ_I * g * h * inner(grad(s), u)

def terminus(**kwargs):
    names = ("velocity", "thickness", "surface")
    u, h, s = itemgetter(*names)(kwargs)

    d = firedrake.min_value(s - h, 0)
    τ_I = ρ_I * g * h**2 / 2
    τ_W = ρ_W * g * d**2 / 2

    ν = firedrake.FacetNormal(mesh)
    return (τ_I - τ_W) * inner(u, ν)

Make the initial thickness and velocity.

In [ ]:
h_0 = firedrake.Function(Q).assign(Constant(100))
h = h_0.copy(deepcopy=True)

In [ ]:
x = firedrake.SpatialCoordinate(mesh)[0]
δu = firedrake.Constant(90.0)
expr = firedrake.as_vector((δu * x / Lx, 0))
u_0 = firedrake.Function(V).interpolate(expr)
u = u_0.copy(deepcopy=True)

Form the momentum balance equation and boundary conditions.

In [ ]:
def smooth_max(a, b, ϵ):
    return (a + b + sqrt((a - b)**2 + ϵ**2)) / 2

In [ ]:
A = Constant(20)
C = Constant(1e-2)

inflow_ids = (1,)
terminus_ids = (2,)
side_wall_ids = (3, 4)

ϵ = Constant(1.0)
s = smooth_max(z_b + h, (1 - ρ_I / ρ_W) * h, ϵ)
#s = firedrake.Function(Q).interpolate(smooth_max(z_b + h, (1 - ρ_I / ρ_W) * h, ϵ))
#s = max_value(z_b + h, (1 - ρ_I / ρ_W) * h)

fields = {
    "velocity": u,
    "thickness": h,
    "surface": s,
}
parameters = {"fluidity": A, "friction": C}
G = (
    viscosity(**fields, **parameters) * dx +
    friction(**fields, **parameters) * dx -
    gravity(**fields, **parameters) * dx -
    terminus(**fields, **parameters) * ds(terminus_ids)
)

F = firedrake.derivative(G, u)

In [ ]:
inflow_bc = firedrake.DirichletBC(V, u_0, inflow_ids)
side_bcs = firedrake.DirichletBC(V.sub(1), 0, side_wall_ids)
bcs = [inflow_bc, side_bcs]

Solve the momentum balance equation.

In [ ]:
logfile = ":mismipp.log"
sparams = {
    "snes_max_it": 200,
    "snes_linesearch_type": "nleqerr",
    "snes_monitor": logfile,
}
fparams = {"quadrature_degree": 8}

params = {
    "bcs": bcs,
    "solver_parameters": sparams,
    "form_compiler_parameters": fparams,
}

firedrake.solve(F == 0, u, **params)

Form the mass balance equation.

In [ ]:
def get_test_function(q):
    z, = ufl.algorithms.extract_coefficients(q)
    w = firedrake.TestFunction(z.function_space())
    return firedrake.replace(q, {z: w})

def mass_balance(**kwargs):
    names = ("thickness", "velocity", "accumulation", "thickness_in")
    h, u, a, h_in = itemgetter(*names)(kwargs)
    η = get_test_function(h)

    ν = firedrake.FacetNormal(mesh)
    F_cells = (Dt(h) * η - inner(h * u, grad(η)) - a * η) * dx
    F_outflow = h * max_value(0, inner(u, ν)) * η * ds
    F_inflow = h_in * min_value(0, inner(u, ν)) * η * ds
    return F_cells + F_outflow + F_inflow

Form the larger space consisting of velocity and thickness.

In [ ]:
Z = V * Q

z = firedrake.Function(Z)
z.sub(0).assign(u)
z.sub(1).assign(h);

New BCs.

In [ ]:
inflow_bc = firedrake.DirichletBC(Z.sub(0), u_0, inflow_ids)
side_bcs = firedrake.DirichletBC(Z.sub(0).sub(1), 0, side_wall_ids)
bcs = [inflow_bc, side_bcs]

Form the coupled problem.

In [ ]:
import ufl

u, h = firedrake.split(z)
v, η = firedrake.TestFunctions(Z)

s = smooth_max(z_b + h, (1 - ρ_I / ρ_W) * h, ϵ)
fields = {
    "velocity": u,
    "thickness": h,
    "surface": s,
}
parameters = {"fluidity": A, "friction": C}
G = (
    viscosity(**fields, **parameters) * dx +
    friction(**fields, **parameters) * dx -
    gravity(**fields, **parameters) * dx -
    terminus(**fields, **parameters) * ds(terminus_ids)
)

F_momentum = ufl.algorithms.expand_derivatives(firedrake.derivative(G, u, v))

a = firedrake.Constant(0.3)
F_mass = mass_balance(thickness=h, velocity=u, accumulation=a, thickness_in=h_0)

F = F_mass + F_momentum

Coupled solver.

In [ ]:
method = irksome.BackwardEuler()
t = Constant(0.0)
timestep = 2.5
dt = Constant(timestep)

fparams = {"quadrature_degree": 8}
params = {
    "bcs": bcs,
    "form_compiler_parameters": fparams,
    "solver_parameters": sparams,
}
solver = irksome.TimeStepper(F, method, t, dt, z, **params)

zs = [z.copy(deepcopy=True)]
final_time = 3600.0
num_steps = int(final_time / timestep)
for step in trange(num_steps):
    solver.advance()
    zs.append(z.copy(deepcopy=True))

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect("equal")
ax.set_axis_off()
colors = firedrake.tripcolor(zs[-1].sub(1), axes=ax)
fig.colorbar(colors, orientation="horizontal");